Подключение библиотек

In [1]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px

# 1) Исследование данных

In [ ]:
data = pd.read_csv(r"sm_lab_data_prepared.csv")

data.head(3)

,device_type,event_id,user_id,user_is_authorized,location_id,create_timestamp,product_id,product_is_pick_up,product_is_pick_point,product_is_delivery,product_is_now,product_price,ret_discount,product_category,product_gender,product_age,brand_lvl,add_to_cart_flag,create_order_flag
0,desktop,cd4192b5-3709-46dd-a601-444dde3e762d,1,0,968,2024-06-02 10:46:21.989,68390,0.0,1.0,0.0,0.0,5280,950,инвентарь,унисекс,взрослые,1,0,0
1,desktop,dd8a2b1a-9268-4930-b414-cfbd9cda16d7,2,0,725,2024-06-08 20:03:52.614,46818,0.0,1.0,1.0,0.0,2599,1300,одежда,женский,взрослые,1,0,0
2,desktop,0416ecb1-e53a-474b-9cbc-ca3009dbaed6,3,0,968,2024-06-19 06:08:03.096,79467,0.0,1.0,1.0,0.0,5599,3471,одежда,мужской,дети,2,0,0


In [7]:
colors_list = px.colors.qualitative.Prism

Количество активных пользователей

In [8]:
active_users = len(data["user_id"].unique())
unauthorized_users = len(data["user_id"][data["user_is_authorized"] == 0].unique())

print(f"Число активных пользователей за время наблюдения - {active_users}. Из них {unauthorized_users} ({round(unauthorized_users / active_users * 100, 1)}%) не авторизованы в момент совершения действия.")

Число активных пользователей за время наблюдения - 2108035. Из них 571707 (27.1%) не авторизованы в момент совершения действия.


In [11]:
len(data["user_id"].unique()) / data.shape[0] * 100

30.114833037594774

Временной промежуток сбора данных

In [18]:
data["create_datetime"] = pd.to_datetime(data["create_timestamp"])
time_delta = max(data["create_datetime"]) - min(data["create_datetime"])

time_delta

Timedelta('20 days 09:20:01.934000')

Сбор данных проводился около 20 дней

Территория нахождения клиентов

In [23]:
locations = data["location_id"].unique()

print(f"С диджитал каналами Спортмастера клиенты взаимодействовали из {len(locations)} различных локаций")

С диджитал каналами Спортмастера клиенты взаимодействовали из 4701 различных локаций


In [28]:
count_top10_locations = data.groupby(by = "location_id")["event_id"].count().sort_values(ascending = False)[:10]
count_top10_locations_keys = [str(location) for location in count_top10_locations.keys()]

fig = go.Figure()

fig.add_trace(go.Bar(x = count_top10_locations_keys, y = count_top10_locations.values, marker_color = colors_list[1]))

fig.update_layout(height = 700, width = 1400)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.update_layout(title_text = "Распределение активности пользователей по территориям нахождения")
fig.update_layout(font_family = "Arial", font_size = 13, title_font_family = "Arial", font_color = "black")
fig.update_xaxes(title_text = "Индентификатор территории")
fig.update_yaxes(title_text = "Количество пользователей")
fig.show()

Продукция

In [47]:
products = data.groupby(by = "product_id")["event_id"].count().sort_values(ascending = False)[:10]
products_str = [str(product) for product in products.keys()]

fig = go.Figure()

fig.add_trace(go.Bar(x = products_str, y = products.values, marker_color = colors_list[1]))

fig.update_layout(height = 700, width = 1400)
fig.update_layout(title_text = "Cамые просматриваемые товары")
fig.update_layout(font_family = "Arial", font_size = 13, title_font_family = "Arial", font_color = "black")
fig.update_xaxes(title_text = "Индентификатор товара")
fig.update_yaxes(title_text = "Число просмотров")

fig.show()

In [4]:
categories_grouped = data[data["create_order_flag"] == 1].groupby(by = "product_category")["product_price"].sum()
categories_grouped.sort_values(ascending = False)

product_category
обувь        703646500
одежда       454275453
инвентарь    347855628
Name: product_price, dtype: int64

Распределение типов устройств, которые используют клиенты

In [29]:
device_types = data["device_type"].value_counts().keys()
values = data["device_type"].value_counts().values

fig = go.Figure()

fig.add_trace(go.Bar(x = device_types, y = values, marker_color = colors_list[1]))

fig.update_layout(height = 700, width = 1400)
fig.update_layout(title_text = "Распределение использования диджитал каналов Спортмастера")
fig.update_layout(font_family = "Arial", font_size = 13, title_font_family = "Arial", font_color = "black")
fig.update_xaxes(title_text = "Канал")
fig.update_yaxes(title_text = "Число пользователей")

fig.show()

Наибольшее количество взаимодействий приходится на приложение на устройствах Android. Наименее популярный канал взаимодействия - десктопная версия сайта.

Распределение значений конверсии продаж по диджитал каналам

*конверсия продаж = (число пользователей, оформивших заказ / число пользователей, поросмотревших карточку товара) * 100(%)*

In [30]:
coeffs = []

for device_type in data["device_type"].unique():
    conversion = data[(data["device_type"] == device_type) & (data["create_order_flag"] == 1)].shape[0] / data[(data["device_type"] == device_type) & (data["create_order_flag"] == 0)].shape[0] * 100
    coeffs.append(conversion)

In [31]:
device_types = data["device_type"].unique()

fig = go.Figure()

fig.add_trace(go.Bar(x = device_types, y = coeffs, marker_color = colors_list[1]))

fig.update_layout(height = 700, width = 1400)
fig.update_layout(xaxis={'categoryorder':'total descending'})
fig.update_layout(title_text = "Распределение значений конверсии продаж по диджитал каналам, %")
fig.update_layout(font_family = "Arial", font_size = 13, title_font_family = "Arial", font_color = "black")
fig.update_xaxes(title_text = "Канал")
fig.update_yaxes(title_text = "Значение конверсии, %")

fig.show()

Несмотря на меньшее количество взаимодействий пользователей с приложением на IOS, чем на Android, конверсия продаж через данный канал выше.

Распределение пола товаров по категориям среди оформленных заказов

In [17]:
cats_gender = pd.pivot_table(data[data["create_order_flag"] == 1], index = ["product_category", "product_gender"], 
                             values = "event_id", aggfunc = "count", sort = True, margins = True)

cats_gender

event_id
product_category product_gender          
инвентарь        женский             4392
                 мужской             5641
                 унисекс            39832
обувь            женский            40096
                 мужской            72138
                 унисекс             9902
одежда           женский            59195
                 мужской            95738
                 унисекс            23369
All                                350303

Среди категорий обувь и одежда было оформлено больше всего заказов товаров для мужчин.

В категории инвентарь лидером продаж являются товары унисекс.

Цены на товары

In [60]:
print(f"Стоимость самого дорого товара - {max(data['product_price'])} руб. Его индентификатор - {data['product_id'][data['product_price'] == max(data['product_price'])].unique()[0]}.")
print(f"Стоимость самого дешёвого товара - {min(data['product_price'])} руб. Его индентификатор - {data['product_id'][data['product_price'] == min(data['product_price'])].unique()[0]}. \n")

print(f"Средняя цена товаров - {round(data['product_price'].unique().mean(), 1)} руб. Медианная - {round(np.median(data['product_price'].unique()), 1)} руб.")

Стоимость самого дорого товара - 991600 руб. Его индентификатор - 90371.
Стоимость самого дешёвого товара - 25 руб. Его индентификатор - 90414. 

Средняя цена товаров - 19034.3 руб. Медианная - 7441.5 руб.
